### Herencia, modulos y mixins

Hemos visto que cuando el método `puts` necesita convertir un objeto en una cadena, llama al método `to_s` de ese objeto. Pero también hemos escrito  clases que no implementan explícitamente `to_s`. 

A pesar de esto, las instancias de estas clases responden exitosamente cuando llamamos a `to_s` en ellas. Cómo funciona esto tiene que ver con la herencia y cómo Ruby usa la herencia para determinar qué método ejecutar cuando envías un mensaje a un objeto. 

La herencia te permite crear una clase que es un refinamiento o especialización de otra clase. Esta clase especializada se denomina `subclase` de la original y la original es una `superclase` de la subclase. La gente también se refiere a esta relación como clases de padres e hijos.

In [1]:
class Padre
  def say_hello
    puts "Hola a  #{self}"
  end
end
p = Padre.new
p.say_hello

class  Hijo < Padre # < less than
end
c = Hijo.new
c.say_hello

Hola a  #<#<Class:0x0000022d29c1e938>::Padre:0x0000022d2951c810>
Hola a  #<#<Class:0x0000022d29c1e938>::Hijo:0x0000022d29517c70>


**Pregunta:** Realiza una explicación del código anterior.

In [ ]:
Se crea el objeto p "Padre" el cual será Padre de la clase Hijo, heredando su método "say_hello"

El método `superclass` devuelve el padre de una clase particular. 

In [2]:
class Padre
end

class Hijo < Padre
end
Hijo.superclass

#<Class:0x0000022d29c1e938>::Padre

Pero, ¿cuál es la superclase de `Padre`?

In [3]:
class Padre
end
Padre.superclass

Object

Si no defines una superclase explícita al definir una clase, Ruby usa automáticamente la clase incorporada `Object` como padre de la clase. Vayamos más allá:

In [4]:
Object.superclass

BasicObject

La clase `BasicObject` es un objeto muy, muy mínimo que se utiliza en ciertos tipos de metaprogramación. ¿Cuál es su padre?

In [6]:
BasicObject.superclass # => nil

Podemos descubrir por qué `to_s` está disponible en casi todos los objetos de Ruby. Resulta que `to_s` está definido en la clase `Object`.  Debido a que `Object` es un ancestro de todas las clases de Ruby excepto `BasicObject`, las instancias de cada clase de Ruby tienen un método `to_s` definido:

In [ ]:
class Persona
  def initialize(nombre)
    @nombre = nombre
  end
end
p = Persona.new("Chalo")
puts p

Podemos sobreescribir  el método `to_s`:

In [11]:
class Persona
    def initialize(nombre)
        @nombre = nombre
    end

    def to_s
        "Persona llamada #{@nombre}"
    end
end

p = Persona.new("Omar")
puts p

Persona llamada Omar


#### Ejemplo 
Imagina que estás trabajando en una aplicación de seguimiento de tareas. Una tarea puede estar en uno de varios estados: puede estar terminada, puede estar iniciada pero incompleta o puede estar definida pero no iniciada. Puede haber otros estados, pero sólo esos tres probablemente sean suficientes para dejar claro el punto. 

Si estás escribiendo código que interactúa con las tareas de este sistema, probablemente tendrás que tener en cuenta el estado de una tarea en tu código. En otras palabras, es probable que siempre escribas código como este: 

In [13]:
def cadena_estados(tarea)
  case tareas.status
  when "realizado" then "Lo he realizado"
  when "iniciado" then "no lo he realizado"
  when "no iniciado" then "ni siquiera he empezado"
  end
end

:cadena_estados

Y estarias cambiando continuamente el estado de una tarea. Esta es una forma de duplicación: si la lista de estados cambia, sería necesario actualizar cada una de estas declaraciones `if` o `case` y  por lo tanto, parece que vale la pena intentar reducir la cantidad de veces que utilizamos esa lógica de conmutación. 

Podemos usar la herencia para crear una jerarquía de clases de estado y luego realizar la lógica de cambio solo una vez:

In [31]:
class Estado
    def self.for(estado)
        case estado
        when "realizado" then Realizado.new()
        when "iniciado" then Iniciado.new()
        when "no iniciado" then Definido.new()
        end
    end

    def done? = false
    def cadena_estados = raise NotImplementedError
end

class Realizado < Estado
    def to_s = "Realizado"
    def done? = true
    def cadena_estados = "Yo soy yo"
end

class Iniciado < Estado
    def to_s = "Iniciado"
    def done? = true
    def cadena_estados = "Yo soy yo"
end

class Definido < Estado
    def to_s = "Definido"
    def done? = true
    def cadena_estados = "Yo soy yo"
end

mensaje1 = Realizado.new("")
mensaje2 = Iniciado.new("")
mensaje3 = Definido.new("")
puts mensaje1, mensaje2, mensaje3

Realizado
Iniciado
Definido


Es un modismo común cuando se utilizan subclases. Una clase principal asume que será subclasificada y llama a un método que espera que sus hijos implementen. Esto permite al padre asumir la peor parte del procesamiento, pero invocar lo que efectivamente son métodos de enlace en subclases para agregar funcionalidad a nivel de aplicación.  

Entonces, en lugar de eso, están los **mixins**, una forma diferente de compartir funcionalidad en el código Ruby. 

### Módulos y espacio de nombres

Los módulos son una forma de agrupar métodos, clases y constantes. Los módulos te brindan dos beneficios principales: 

- Los módulos proporcionan un espacio de nombres y evitan conflictos de nombres. 

- Los módulos se pueden incluir en otras clases, una facilidad conocida como mixin. 

Los módulos definen un espacio de nombres, una zona de pruebas en la que los métodos y constantes pueden jugar sin tener que preocuparse de que otros métodos y constantes los molesten. 

Veamos los siguientes ejemplos.

In [ ]:
module Trig
  PI = 3.141592654
  def self.sin(x) # Sin hace referencia a la funcion trigonometrica
    # ..
  end
  def self.cos(x)
    # ..
  end
end

In [ ]:
module Moral
  MUY_MALO= 0
  MALO = 1
  def self.sin(maldad) # Acá hace referencia a la palabra "Pecado"
  # ...
  end
end

Los nombres de los módulos son como los nombres de las clases, ambos son constantes globales con una letra mayúscula inicial. Las definiciones de sus métodos también son similares: los métodos de los módulos se definen igual que los métodos de clase, utilizando la sintaxis `def self.method_name`. 

Si un tercer programa quiere usar estos módulos, simplemente puedes cargar los dos archivos (usando la instrucción `require` o `require_relative` de Ruby). 

In [ ]:
#require_relative "trig"
#require_relative "moral"
#y = Trig.sin(Trig::PI / 4)
#haciendolo_mal = Moral.sin(Moral::MUY_MALO)

**Ejercicio:** Comprueba estos ejemplos.

In [ ]:
# Tu respuesta

Al igual que con los métodos de clase, se llama a un método de módulo precediendo su nombre con el nombre del módulo y un punto. Como resultado, ahora se puede acceder a un método como `Trig.sin` y el otro es `Moral.sin` y los nombres ya no entran en conflicto. 

Se hace referencia a las constantes del módulo utilizando el nombre del módulo seguido de dos dos puntos, lo que se denomina operador de resolución de alcance, por lo que en este ejemplo, `Trig::PI` y `Moral::MUY_MALO`. 

### Mixins

Los módulos tienen otro uso maravilloso. Pueden proporcionar una alternativa a la herencia como forma de ampliar las clases. Esta instalación a veces se denomina **mixin**. Los mixins permiten algo muy parecido a la herencia múltiple en otros lenguajes. 
 

In [32]:
module Debug
  def quien_soy?
    "#{self.class.name} (id: #{self.object_id}): #{self.name}"
  end
end

class Fonografo
  include Debug
  attr_reader :name

  def initialize(nombre)
    @name = nombre
  end
# ...
end

class OchoPistas
  include Debug
  attr_reader :name

  def initialize(nombre)
    @name = nombre
  end
# ...
end

fonografo = Fonografo.new("West End Blues")
ocho_pistas = OchoPistas.new("Surrealistic Pillow")
fonografo.quien_soy?


"#<Class:0x0000022d29c1e938>::Fonografo (id: 1740): West End Blues"

**Pregunta**: Realiza una explicación del código anterior.

In [ ]:
Se incluye el modulo debug en las clases OchoPistas y Fonografo de tal manera que se puede usar el método "quiensoy?"

Mixins te brinda una forma controlada de agregar funcionalidad a las clases. Sin embargo, su verdadero poder surge cuando el código en el mixin puede hacer suposiciones sobre el código de la clase que lo usa y luego puede interactuar con ese código. 

Ruby utiliza ampliamente el comportamiento mixin en la biblioteca estándar. Muchos de los comportamientos que hemos visto que están disponibles para todos los objetos en realidad están definidos en un módulo llamado `Kernel` que está incluido en `Object` y por tanto en todos los objetos. Métodos como `puts`, `p`, `lambda`, `proc` y muchos más se agregan a los objetos mediante el comportamiento mixin. 

#### Módulo Comparable

El módulo estándar de Ruby `Comparable` es otro gran ejemplo de mixin, pero hace una suposición sobre las clases que lo usan. Incluir `Comparable` como mixin agrega los operadores de comparación (`<, <=, ==, >=` y `>`), así como el método `between?` a una clase. Para que estos métodos funcionen, `Comparable` supone que cualquier clase que los utilice define el método `<=>`. 

Como programador, puedes definir un método  `<=>` incluir `Comparable` y obtener seis funciones de comparación. 

Tomemos una clase `Persona`. Haremos que las personas sean comparables según sus nombres: 

In [45]:
class Person
  include Comparable
  attr_reader :name
  def initialize(name)
    @name = name
  end

  def to_s
    "#{@name}"
  end
    
  def<=>(other)
    self.name <=> other.name
  end
end

p1 = Persona.new("M")
p2 = Persona.new("G")
p3 = Persona.new("L")

if p1 > p2
    puts "nombre #{p1.name} > nombre #{p2.name}"
end

puts "Lista ordenada:"
puts [p1,p2,p3].sort

NoMethodError: undefined method `>' for #<#<Class:0x0000022d29c1e938>::Persona:0x0000022d2a784d90 @nombre="M">

Incluimos `Comparable` en la clase `Persona` y luego definimos un método `<=>`. Luego pudimos realizar comparaciones (como `p1 > p2`) e incluso ordenar una serie de objetos `Persona`.

### Herencia y mixins

Algunos lenguajes orientados a objetos (como C++ o Python) admiten herencia múltiple, donde una clase puede tener más de un padre inmediato, heredando la funcionalidad de cada uno. Aunque poderosa, esta técnica puede ser peligrosa porque la jerarquía de herencia puede volverse ambigua.

Otros lenguajes, como Java, JavaScript y C#, admiten herencia única. Aquí, una clase sólo puede tener un padre inmediato. Aunque es más limpia (y más fácil de implementar), la herencia única también tiene desventajas: en el mundo real, los objetos a menudo heredan atributos de múltiples fuentes.

Ruby ofrece un compromiso interesante, brindando la simplicidad de la herencia única y el poder de la herencia múltiple. Una clase Ruby tiene solo un padre directo, por lo que Ruby es un lenguaje de herencia único. Sin embargo, las clases de Ruby pueden incluir la funcionalidad de cualquier número de mixins (un mixin es como una definición de clase parcial). Esto proporciona una capacidad controlada similar a la herencia múltiple sin ninguno de los inconvenientes.

#### extend y prepend

Ruby proporciona dos mecanismos para mezclar el comportamiento del módulo que están relacionados a `include` pero combinan el módulo y la clase de manera diferente. El comportamiento de `include` es agregar los métodos del módulo como métodos de instancia a la clase en la que se incluye el módulo, y hacer que esos métodos del módulo se busquen después de que se verifique la clase misma en busca de un método.

Ruby también proporciona `extend`. El comportamiento de `extend` es agregar los métodos directamente al receptor de `extend` en lugar de hacerlo como métodos de instancia de una clase. El efecto de `extend` es agregar los métodos del módulo como métodos de clase.

Ruby también proporciona `prepend`. El comportamiento de `prepend` es el mismo que `include` excepto que un método en un módulo `prepend` se ejecuta antes que un método del mismo nombre en la clase. Normalmente, el método del módulo `prepend` llama a `super` para que también se llame al método de la clase. 

Los `prepend` se utilizan a menudo para agregar registros u otra información logística a las clases.

**Revisa:** https://docs.ruby-lang.org/en/3.2/Module.html.

## Metaprogramación

La programación se trata de construir capas de abstracciones. A medida que resuelves problemas, estás construyendo puentes desde el implacable y mecánico mundo del silicio hasta el mundo más ambiguo y fluido que habitamos. Algunos lenguajes de programación, como C, están cerca de la máquina. La distancia entre el código C y el dominio de la aplicación puede ser grande.

Otros lenguajes (tal vez Ruby) proporcionan abstracciones de nivel superior y, por lo tanto, te permiten comenzar a codificar más cerca del dominio de aplicación. Por esta razón, la mayoría de la gente considera que un lenguaje de nivel superior es un mejor punto de partida para el desarrollo de aplicaciones.

Pero cuando realiza una metaprogramación, ya no estás limitado al conjunto de abstracciones integradas en tu lenguaje de programación. En su lugar, puedes crear nuevas abstracciones que se integran en el lenguaje anfitrión. De hecho, estás creando un nuevo lenguaje de programación de dominio específico, uno que te permite expresar los conceptos que necesitas para resolver tu problema particular.

#### Objetos y clases

Internamente, un objeto Ruby tiene tres componentes: un conjunto de indicadores, algunas variables de instancia y una clase asociada. 
Una clase Ruby contiene todas las cosas que tiene un objeto más un conjunto de definiciones de métodos y una referencia a una superclase (que es en sí misma otra clase). (Una clase Ruby es en sí misma una instancia de la clase `Class`). 

Veamos cómo esa estructura se presta a la metaprogramación en Ruby. 

 ### Llamadas a métodos y 'self' 
 
Ruby tiene un concepto del objeto actual. La variable incorporada de solo lectura `self` hace referencia a este objeto actual.

El valor del `self`  tiene dos roles importantes en un programa Ruby en ejecución. Primero, `self`  controla automáticamente cómo Ruby encuentra variables de instancia. 

En segundo lugar, `self` juega un papel vital en la llamada al método. En Ruby, cada llamada a un método es un mensaje que se pasa a algún objeto. Este objeto se llama `receptor` de la llamada. Cuando se realiza una llamada como `items.size`, el objeto en el lado izquierdo del punto (aquí al que hace referencia la variable `items`) es el receptor y `size` es el método a invocar. 

Cuando se realiza una llamada a un método con un receptor explícito (por ejemplo, invocando `items.size`), el valor de `self` cambia durante la duración de la llamada. Antes de iniciar el proceso de búsqueda de métodos, Ruby configura a `sef` como el receptor explícito (el objeto al que hacen referencia los elementos en este caso). Luego, después de que regresa la llamada, Ruby se restaura `self` al valor que tenía antes de la llamada. 

Veamos cómo funciona esto en la práctica. Aquí tienes un programa sencillo 

In [ ]:
class Test
  def one
    @var = 99
    two
  end
  def two
    puts @var
  end
end

t = Test.new
t.one

**Pregunta:** Realiza una explicación del código anterior.

In [ ]:
# Tus respuestas.

### Definiciones de clases y self

Hemos visto que llamar a un método con un receptor explícito cambia `self`. Quizás sea sorprendente que `self` también cambie dentro de una definición de clase o módulo. 

Esto es una consecuencia del hecho de que las definiciones de clases son en realidad código ejecutable en Ruby, si podemos ejecutar código, necesitamos tener un objeto actual.  

In [ ]:
puts "Antes de la deficion de clase"
puts "self = #{self}\n"
class Test
puts "En la definicion de clase"
puts "self = #{self}"
puts "Clase de self = #{self.class}\n"
end
puts "Despues de la deficion de clase"
puts "self = #{self}"

Dentro de una definición de clase o módulo, `self` se establece en el objeto de la clase o módulo que se define. Esto significa que las variables de instancia establecidas dentro de una definición de clase o módulo estarán disponibles para los métodos de clase o módulo (porque `self` será el mismo cuando se definan las variables y cuando se ejecuten los métodos): 

In [ ]:
class Test
  @var = 99
  def self.valor_de_var
    @var
  end
end
Test.valor_de_var

El hecho de que `self` se establezca en la clase durante una definición de clase resulta ser una decisión espectacularmente elegante, pero para ver por qué, primero tendremos que echar un vistazo a los **singletons**.

### Definición de métodos singleton

Ruby te permite definir métodos que son específicos de un objeto en particular. Estos se denominan métodos singleton.

Aquí hay un objeto de cadena y una llamada a un método normal que no es singleton:

In [ ]:
animal = "cat"
puts animal.upcase

Esta llamada da como resultado la siguiente estructura de objetos_

<img src="Imagenes/Meta1.png" alt="drawing" width="600"/>

**Pregunta:** Explica que sucede en el gráfico anterior.

In [ ]:
# Tu respuesta

Definiremos un método singleton en la cadena a la que hace referencia `animal`.

In [ ]:
animal = "cat"
def animal.speak
  puts "El #{self} dice miaow"
end

animal.speak
puts animal.upcase

**Pregunta:** Explica que sucede en el código anterior.

In [ ]:
# Tu respuesta

Cada objeto en Ruby tiene el potencial de tener su propia clase singleton. Cuando defines una clase singleton, Ruby crea esa clase anónima y la convierte en la clase singleton de ese objeto. Puedes acceder a esa clase singleton a través del método `singleton_class` y puede obtener una lista de métodos definidos allí con `singleton_methods`. 

In [ ]:
animal = "cat"
def animal.speak
  puts "El #{self} dice miaow"
end

animal.speak
puts animal.class
puts animal.singleton_class
puts animal.singleton_methods

Si un objeto tiene una clase singleton, ese es el primer lugar donde Ruby busca objetos. Es como si Ruby hiciera de String (que era la clase original de "cat") la superclase de la clase singleton. La imagen se ve así:

<img src="Imagenes/Meta2.png" alt="drawing" width="600"/>

**Pregunta:** Explica que sucede en el gráfico anterior.

In [ ]:
# Tu respuesta

### Singletons y clases 

Anteriormente dijimos que dentro de una definición de clase, `self` se establece en el objeto de clase que se define. Resulta que esta es la base de uno de los aspectos más elegantes del modelo de objetos de Ruby. 

Recuerda que podemos definir métodos de clase en Ruby usando cualquiera de las formas `def self.xxx` o (más raramente) `def ClassName.xxx`: 

In [ ]:
class Dave
  def self.class_method_one
    puts "Metodo de clase 1"
  end
  def Dave.class_method_two
    puts "Metodo de clase 2"
  end
end

Dave.class_method_one
Dave.class_method_two

**Pregunta:** ¿por qué las dos formas son idénticas?

In [ ]:
# Tu respuesta

**Pregunta:** ¿que sucede en la siguiente situación?

<img src="Imagenes/Meta3.png" alt="drawing" width="650"/>

In [ ]:
## Tu respuesta

**Pregunta**: ¿que sucede en la siguiente situación?

<img src="Imagenes/Meta4.png" alt="drawing" width="650"/>

In [ ]:
# Tu respuesta

Ahora podemos unir los dos usos de `self`, al objeto actual. Hablamos sobre cómo se buscan las variables de instancia en `self` y hablamos sobre cómo los métodos singleton definidos en `self` se convierten en métodos de clase. Usemos estos hechos para acceder a variables de instancia para objetos de clase:

In [ ]:
class Test
  @var = 99
  def self.var
    @var
  end
  def self.var=(value)
    @var = value
  end
end
puts "Valor original = #{Test.var}"
Test.var = "cat"
puts "Nuevo valor = #{Test.var}"

Ciertos programadores intentan establecer variables de instancia en la definición de clase (como hicimos con `@var` en el código anterior) y luego intentan acceder a estas variables desde métodos de instancia. Como ilustra el código, esto no funcionará porque las variables de instancia están asociadas con `self` en su contexto actual. 

En el contexto de las variables de instancia definidas en el cuerpo de la clase, `self` es la clase y por lo tanto, las variables de instancia definidas en el cuerpo de la clase fuera de los métodos están asociadas con el objeto de la clase, no con las instancias de la clase. 

#### Módulos y mixins

Sabes que cuando incluyes un módulo en una clase Ruby, los métodos de instancia de ese módulo quedan disponibles como métodos de instancia de la clase, así:


In [ ]:
module Logger
  def log(msg)
    STDERR.puts Time.now.strftime("%H:%M:%S: ") + "#{self} (#{msg})"
  end
end

class Song
  include Logger
end

s = Song.new
s.log("creado")

**Pregunta**: Explica que sucede en el código anterior.

In [ ]:
# Tu respuesta

Ruby implementa  `include` de manera muy simple: el módulo que incluye se agrega como un antepasado de la clase que se está definiendo. Es como si el módulo fuera el padre de la clase en la que está mezclado. Y ese sería el final de la descripción excepto por una pequeña problema. Debido a que el módulo se inyecta en la cadena de superclases, él mismo debe mantener un vínculo con la clase principal original.

Si no fuera así, no habría forma de atravesar la cadena de superclases para buscar métodos. Sin embargo, puedes mezclar el mismo módulo en muchas clases diferentes, y esas clases podrían tener cadenas de superclases totalmente diferentes. Si hubiera solo un objeto de módulo que mezcláramos con todas estas clases, no habría forma de realizar un seguimiento de las diferentes superclases para cada una.

¿Qué hace Ruby en ese caso?.